# Propositions Chunking

### Tổng quan

Đoạn mã này triển khai phương pháp *proposition chunking*, dựa trên [nghiên cứu của Tony Chen và cộng sự](https://doi.org/10.48550/arXiv.2312.06648). Hệ thống phân tích văn bản đầu vào thành các *propositions* (mệnh đề) mang tính chất nguyên tử, thực tế, độc lập và súc tích, mã hóa các *propositions* này vào một *vectorstore*, có thể được sử dụng sau này để truy xuất.

### Các thành phần chính

1.  **Document Chunking:** Chia tài liệu thành các phần có thể quản lý được để phân tích.
2.  **Proposition Generation:** Sử dụng LLMs để phân tích các *document chunks* thành các *propositions* mang tính thực tế và độc lập.
3.  **Proposition Quality Check:** Đánh giá các *propositions* được tạo ra dựa trên độ chính xác, rõ ràng, đầy đủ và súc tích.
4.  **Embedding and Vector Store:** *Embedding* cả *propositions* và các *chunks* lớn hơn của tài liệu vào một *vector store* để truy xuất hiệu quả.
5.  **Retrieval and Comparison:** Kiểm tra hệ thống truy xuất với các kích thước truy vấn khác nhau và so sánh kết quả từ mô hình dựa trên *proposition* với mô hình dựa trên *chunk* lớn hơn.

<img src="images/proposition_chunking.svg" alt="Reliable-RAG" width="600">

### Động lực

Động lực đằng sau phương pháp *proposition chunking* là xây dựng một hệ thống phân tích tài liệu văn bản thành các *propositions* súc tích, thực tế để truy xuất thông tin chi tiết và chính xác hơn. Việc sử dụng *propositions* cho phép kiểm soát chi tiết hơn và xử lý tốt hơn các truy vấn cụ thể, đặc biệt là để trích xuất kiến thức từ các văn bản chi tiết hoặc phức tạp. Việc so sánh giữa việc sử dụng các *proposition chunks* nhỏ hơn và các *document chunks* lớn hơn nhằm đánh giá hiệu quả của việc truy xuất thông tin chi tiết.

### Chi tiết phương pháp

1.  **Loading Environment Variables:** Mã bắt đầu bằng cách tải các biến môi trường (ví dụ: khóa API cho dịch vụ LLM) để đảm bảo hệ thống có thể truy cập các tài nguyên cần thiết.

2.  **Document Chunking:**
    -   Tài liệu đầu vào được chia thành các phần nhỏ hơn (*chunks*) bằng cách sử dụng `RecursiveCharacterTextSplitter`. Điều này đảm bảo rằng mỗi *chunk* có kích thước quản lý được để LLM xử lý.

3.  **Proposition Generation:**
    -   Các *propositions* được tạo ra từ mỗi *chunk* bằng cách sử dụng LLM (trong trường hợp này, "llama-3.1-70b-versatile"). Đầu ra được cấu trúc dưới dạng danh sách các câu lệnh thực tế, độc lập có thể hiểu được mà không cần ngữ cảnh bổ sung.

4.  **Quality Check:**
    -   Một LLM thứ hai đánh giá chất lượng của các *propositions* bằng cách chấm điểm chúng dựa trên độ chính xác, rõ ràng, đầy đủ và súc tích. Các *propositions* đáp ứng các ngưỡng yêu cầu trong tất cả các danh mục sẽ được giữ lại.

5.  **Embedding Propositions:**
    -   Các *propositions* vượt qua kiểm tra chất lượng được *embedding* vào một *vector store* bằng mô hình `OllamaEmbeddings`. Điều này cho phép truy xuất các *propositions* dựa trên sự tương đồng khi có truy vấn.

6.  **Retrieval and Comparison:**
    -   Hai hệ thống truy xuất được xây dựng: một hệ thống sử dụng các *proposition-based chunks* và một hệ thống khác sử dụng các *document chunks* lớn hơn. Cả hai đều được kiểm tra với một số truy vấn để so sánh hiệu suất và độ chính xác của kết quả trả về.

### Lợi ích

-   **Granularity (Độ chi tiết):** Bằng cách phân tích tài liệu thành các *propositions* thực tế nhỏ, hệ thống cho phép truy xuất rất cụ thể, giúp trích xuất câu trả lời chính xác từ các tài liệu lớn hoặc phức tạp dễ dàng hơn.
-   **Quality Assurance (Đảm bảo chất lượng):** Việc sử dụng LLM kiểm tra chất lượng đảm bảo rằng các *propositions* được tạo ra đáp ứng các tiêu chuẩn cụ thể, cải thiện độ tin cậy của thông tin được truy xuất.
-   **Flexibility in Retrieval (Tính linh hoạt trong truy xuất):** Việc so sánh giữa truy xuất dựa trên *proposition* và truy xuất dựa trên *chunk* lớn hơn cho phép đánh giá sự đánh đổi giữa độ chi tiết và ngữ cảnh rộng hơn trong kết quả tìm kiếm.

### Triển khai

1.  **Proposition Generation:** LLM được sử dụng kết hợp với một *prompt* tùy chỉnh để tạo ra các câu lệnh thực tế từ các *document chunks*.
2.  **Quality Checking:** Các *propositions* được tạo ra được chuyển qua một hệ thống chấm điểm đánh giá độ chính xác, rõ ràng, đầy đủ và súc tích.
3.  **Vector Store Integration:** Các *propositions* được lưu trữ trong một *vector store* FAISS sau khi được *embedding* bằng một mô hình *embedding* được đào tạo trước, cho phép tìm kiếm và truy xuất dựa trên sự tương đồng hiệu quả.
4.  **Query Testing:** Nhiều truy vấn kiểm tra được thực hiện đối với các *vector stores* (*proposition-based* và *chunks* lớn hơn) để so sánh hiệu suất truy xuất.

### Tóm tắt

Đoạn mã này trình bày một phương pháp mạnh mẽ để phân tích một tài liệu thành các *propositions* độc lập bằng cách sử dụng LLMs. Hệ thống thực hiện kiểm tra chất lượng trên mỗi *proposition*, *embedding* chúng trong một *vector store* và truy xuất thông tin phù hợp nhất dựa trên truy vấn của người dùng. Khả năng so sánh các *propositions* chi tiết với các *document chunks* lớn hơn cung cấp thông tin chi tiết về phương pháp nào mang lại kết quả chính xác hoặc hữu ích hơn cho các loại truy vấn khác nhau. Phương pháp này nhấn mạnh tầm quan trọng của việc tạo và truy xuất *propositions* chất lượng cao để trích xuất thông tin chính xác từ các tài liệu phức tạp.


In [4]:
# Basic setup
import os
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)

embedding_model='bge-m3:latest'
embeddings = OllamaEmbeddings(model=embedding_model)

In [5]:
# Load pdf file and split file to chunks
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

PDF_FILE_PATH='law_gtdb/data/law_gtdb_35.pdf'
chunk_size = 200
chunk_overlap = 50

loader = PyPDFLoader(PDF_FILE_PATH)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
  chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

doc_split = text_splitter.split_documents(documents)

In [7]:
doc_split[0].page_content

'CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 3 \n \n \nQUỐC HỘI \n \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \nLuật số: 35/2024/QH15  \n \nLUẬT \nĐƯỜNG BỘ'

In [8]:
doc_split[0].metadata

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '2024-09-20T10:02:58+07:00',
 'moddate': '2024-09-20T10:02:58+07:00',
 'source': 'law_gtdb/data/law_gtdb_35.pdf',
 'total_pages': 69,
 'page': 0,
 'page_label': '1'}

In [9]:
for i, doc in enumerate(doc_split):
  doc.metadata['chunk_id'] = i+1

doc_split[0].metadata

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '2024-09-20T10:02:58+07:00',
 'moddate': '2024-09-20T10:02:58+07:00',
 'source': 'law_gtdb/data/law_gtdb_35.pdf',
 'total_pages': 69,
 'page': 0,
 'page_label': '1',
 'chunk_id': 1}

### Create Propositions

In [10]:
from typing import List
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from pydantic import BaseModel, Field

# Data model
class GeneratePropositions(BaseModel):
    """List of all the propositions in a given document"""

    propositions: List[str] = Field(
        description="List of propositions (factual, self-contained, and concise information)"
    )

structured_llm= llm.with_structured_output(GeneratePropositions)


# Few shot prompting --- We can add more examples to make it good
proposition_examples = [
    {"document": 
        "In 1969, Neil Armstrong became the first person to walk on the Moon during the Apollo 11 mission.", 
     "propositions": 
        "['Neil Armstrong was an astronaut.', 'Neil Armstrong walked on the Moon in 1969.', 'Neil Armstrong was the first person to walk on the Moon.', 'Neil Armstrong walked on the Moon during the Apollo 11 mission.', 'The Apollo 11 mission occurred in 1969.']"
    },
]

example_proposition_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{document}"),
        ("ai", "{propositions}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_proposition_prompt,
    examples = proposition_examples,
)

# Prompt
system = """Please break down the following text into simple, self-contained propositions. Ensure that each proposition meets the following criteria:

    1. Express a Single Fact: Each proposition should state one specific fact or claim.
    2. Be Understandable Without Context: The proposition should be self-contained, meaning it can be understood without needing additional context.
    3. Use Full Names, Not Pronouns: Avoid pronouns or ambiguous references; use full entity names.
    4. Include Relevant Dates/Qualifiers: If applicable, include necessary dates, times, and qualifiers to make the fact precise.
    5. Contain One Subject-Predicate Relationship: Focus on a single subject and its corresponding action or attribute, without conjunctions or multiple clauses."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        few_shot_prompt,
        ("human", "{document}"),
    ]
)

proposition_generator = prompt | structured_llm

In [ ]:
{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '2024-09-20T10:02:58+07:00',
 'moddate': '2024-09-20T10:02:58+07:00',
 'source': 'law_gtdb/data/law_gtdb_35.pdf',
 'total_pages': 69,
 'page': 0,
 'page_label': '1',
 'chunk_id': 1}

In [11]:
doc_split[0].metadata['source']

'law_gtdb/data/law_gtdb_35.pdf'

In [ ]:
from tqdm import tqdm
from langchain_core.documents import Document

propositions = [] # Store all the propositions from the document

for i in tqdm(range(len(doc_split))):
    response = proposition_generator.invoke({"document": doc_split[i].page_content}) # Creating proposition
    try:
      for proposition in response.propositions:
        propositions.append(
          Document(
            page_content=proposition, 
            metadata={"source": doc_split[i].metadata['source'], 
                      "total_pages": doc_split[i].metadata['total_pages'],
                      "page": doc_split[i].metadata['page'],
                      "page_label": doc_split[i].metadata['page_label'],
                      "chunk_id": i+1}
            )
          )
    except:
      print("Parser error ")
      continue

  4%|▎         | 33/930 [12:21<4:38:07, 18.60s/it]

Parser error 


 10%|▉         | 90/930 [34:31<5:19:46, 22.84s/it]

Parser error 


 12%|█▏        | 114/930 [44:45<4:10:17, 18.40s/it]

...

 62%|██████▏   | 572/930 [3:46:42<2:10:14, 21.83s/it]

Parser error 


 65%|██████▌   | 608/930 [4:00:45<2:36:14, 29.11s/it]

Parser error 


 66%|██████▌   | 616/930 [4:03:38<1:54:23, 21.86s/it]